# 🔍 Data Reconciliation - Simple Test

## What This Notebook Does:
1. **Creates 2 tables** (source and target)
2. **Inserts test data** (with intentional differences)
3. **Runs reconciliation** (using config file)
4. **Shows results** (summary and details)

## Expected Results:
- ❌ **1 Missing**: David (customer_id=4) in source but not in target
- ❌ **1 Extra**: Eve (customer_id=5) in target but not in source
- ❌ **1 Mismatch**: Bob's city (SF vs NYC)
- ✅ **2 Matched**: Alice and Charlie

## How to Run:
1. **Run Cells 1-6** to create tables and insert data
2. **Run Cell 7** to setup parameters
3. **Run Cell 8** to execute reconciliation
4. **Run Cells 9-10** to view results

## Parameters (Cell 7):
- **recon_name**: Select which reconciliation to run (ALL, customer_recon, product_recon)
- **config_path**: Path to YAML config file

In [0]:
%pip install pyyaml

In [0]:
dbutils.library.restartPython()

In [0]:
%sql
-- Create source table
CREATE OR REPLACE TABLE workspace.default.customer_source (
  customer_id INT,
  customer_name STRING,
  email STRING,
  city STRING
)

In [0]:
%sql
-- Create target table (same schema)
CREATE OR REPLACE TABLE workspace.default.customer_target (
  customer_id INT,
  customer_name STRING,
  email STRING,
  city STRING
)

In [0]:
%sql
-- Insert data into source table
INSERT INTO workspace.default.customer_source VALUES
  (1, 'Alice', 'alice@example.com', 'LA'),
  (2, 'Bob', 'bob@example.com', 'SF'),
  (3, 'Charlie', 'charlie@example.com', 'LA'),
  (4, 'David', 'david@example.com', 'Chicago');

SELECT * FROM workspace.default.customer_source;

In [0]:
%sql
-- Insert data into target table (with differences)
INSERT INTO workspace.default.customer_target VALUES
  (1, 'Alice', 'alice@example.com', 'LA'),
  (2, 'Bob', 'bob@example.com', 'NYC'),  -- Mismatch: city changed
  (3, 'Charlie', 'charlie@example.com', 'LA'),
  (5, 'Eve', 'eve@example.com', 'Boston');  -- Extra record
-- Missing: David (customer_id 4)

SELECT * FROM workspace.default.customer_target;

In [0]:
# Create parameter widgets
dbutils.widgets.dropdown(
    "recon_name",
    "ALL",
    ["ALL", "customer_recon", "product_recon"],
    "Select Reconciliation"
)

dbutils.widgets.text(
    "config_path",
    "/Workspace/Users/mohu.tera@gmail.com/recon/configs/recon_config.yaml",
    "Config File Path"
)

print("✓ Widgets created")
print(f"  Reconciliation: {dbutils.widgets.get('recon_name')}")
print(f"  Config: {dbutils.widgets.get('config_path')}")

In [0]:
import sys
sys.path.append("/Workspace/Users/mohu.tera@gmail.com/recon")

from utils.recon_runner import run_reconciliation

# Get parameters
recon_name = dbutils.widgets.get("recon_name")
config_path = dbutils.widgets.get("config_path")

# Run reconciliation
if recon_name == "ALL":
    results = run_reconciliation(spark, config_path)
else:
    results = run_reconciliation(spark, config_path, [recon_name])

print(f"\n{'='*80}")
print(f"✅ RECONCILIATION COMPLETE")
print(f"{'='*80}")
print(f"Total: {results['total']}")
print(f"Passed: {results['passed']}")
print(f"Failed: {results['failed']}")
print(f"{'='*80}")

In [0]:
%sql
-- View reconciliation results
SELECT 
    recon_name,
    status,
    source_count,
    target_count,
    match_rate,
    missing_count,
    extra_count,
    mismatch_count,
    summary,
    execution_time
FROM workspace.default.recon_results
ORDER BY execution_time DESC
LIMIT 10

In [0]:
%sql
-- View detailed mismatches
SELECT 
    recon_name,
    issue_type,
    record_keys,
    details,
    execution_time
FROM workspace.default.recon_details
ORDER BY execution_time DESC
LIMIT 20